In [ ]:
import os
os.environ["HF_HOME"] = "~/scratch/hf-cache"
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

with torch.no_grad():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = "YOUR HF TOKEN", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/hice1/tkaple3/.conda/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==((====))==  Unsloth 2024.11.6: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA H100 80GB HBM3. Max memory: 79.097 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 9.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 6/6 [05:56<00:00, 59.47s/it]


In [21]:
from bert_score import BERTScorer
from langchain.docstore.document import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

In [3]:
my_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [36]:
import time
start_time = time.time()
with torch.no_grad():
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        my_prompt.format("""
        You are a WikiData Expert for Hindi and English. Generate more Template sentences like the ones given in the example below for the Politician occupation.
        The content between the curly bracket represents the property of wikidata translated into Hindi. Use the Properties for politician to generate similar template sentences.
        Template Sentences for Scientist:
        ["{{Scientist}} \u090f\u0915 \u092a\u094d\u0930\u0938\u093f\u0926\u094d\u0927 {{\u0935\u094d\u092f\u0935\u0938\u093e\u092f}} {alivestatus/wgop} \u091c\u093f\u0928\u0915\u093e \u091c\u0928\u094d\u092e {{\u091c\u0928\u094d\u092e \u0924\u093f\u0925\u093f}} \u0915\u094b {{\u091c\u0928\u094d\u092e \u0938\u094d\u0925\u093e\u0928}} \u0926\u0947\u0936 \u092e\u0947\u0902 \u0939\u0941\u0906 {alivestatus} |", "{{\u0928\u093e\u0917\u0930\u093f\u0915\u0924\u093e}} \u092e\u0947\u0902 \u092a\u0948\u0926\u093e {\u0939\u0941\u090f/\u0939\u0941\u0908} {{Scientist}} {{\u092e\u093e\u0924\u0943\u0938\u0902\u0938\u094d\u0925\u093e}} {\u0915\u0947/\u0915\u0940} \u092a\u0942\u0930\u094d\u0935 \u091b\u093e\u0924\u094d\u0930 {alivestatus/wgop} \u0914\u0930 \u0906\u0917\u0947 \u091a\u0932\u0915\u0947 \u0909\u0928\u094d\u0939\u094b\u0902\u0928\u0947 {{\u0936\u0948\u0915\u094d\u0937\u093f\u0915 \u0926\u0930\u094d\u091c\u093e/\u0909\u092a\u093e\u0927\u093f}} \u0915\u0940 \u0921\u093f\u0917\u094d\u0930\u0940 \u092d\u0940 \u092a\u094d\u0930\u093e\u092a\u094d\u0924 \u0915\u0940 |", "{{Scientist}} \u0915\u093e \u0915\u093e\u0930\u094d\u092f\u0938\u094d\u0925\u0932 {{\u0915\u093e\u0930\u094d\u092f \u0938\u094d\u0925\u0932}} {alivestatus}, \u0914\u0930 \u0935\u0939 {{\u0928\u093f\u092f\u094b\u0915\u094d\u0924\u093e}} \u092e\u0947\u0902 \u090f\u0915 {{\u092a\u0926 \u092a\u0930 \u0906\u0938\u0940\u0928}} \u0915\u0947 \u0930\u0942\u092a \u092e\u0947\u0902 \u092d\u0940 \u0915\u093e\u0930\u094d\u092f\u0930\u0924 {alivestatus/wgop} |", "{{Scientist}} \u0915\u0947 \u0936\u093f\u0915\u094d\u0937\u0915 {{\u0915\u0947 \u091b\u093e\u0924\u094d\u0930 }} {alivestatus/wgok} \u0914\u0930 \u0935\u0939 {{\u091b\u093e\u0924\u094d\u0930}} {\u0915\u0947/\u0915\u0940} \u0936\u093f\u0915\u094d\u0937\u0915 \u092d\u0940 {alivestatus/wgop} \u0914\u0930 \u0907\u0938\u0915\u0947 \u0906\u0932\u093e\u0935\u093e \u0909\u0928\u0915\u0947 \u0921\u0949\u0915\u094d\u091f\u0930\u0947\u091f \u090f\u0921\u0935\u093e\u0907\u091c\u0930\u094d {{\u0921\u0949\u0915\u094d\u091f\u0930\u0947\u091f \u0938\u0932\u093e\u0939\u0915\u093e\u0930}} {alivestatus/wgok} \u0914\u0930 \u0935\u0939 \u0938\u094d\u0935\u092f\u0902 {{\u0921\u0949\u0915\u094d\u091f\u0930\u0947\u091f \u091b\u093e\u0924\u094d\u0930}} {\u0915\u0947/\u0915\u0940} \u0921\u0949\u0915\u094d\u091f\u0930\u0947\u091f \u090f\u0921\u0935\u093e\u0907\u091c\u0930\u094d \u092d\u0940 {alivestatus/wgop} |", "{{Scientist}} \u0915\u094b {{Reason}} \u0915\u0947 \u0932\u093f\u090f {{Reason_Award}} \u0938\u0947 \u0938\u092e\u094d\u092e\u093e\u0928\u093f\u0924 \u0915\u093f\u090f \u091c\u093e\u0928\u0947 \u0915\u0947 \u0938\u093e\u0925-\u0938\u093e\u0925 {{\u092a\u0941\u0930\u0938\u094d\u0915\u093e\u0930 \u092a\u094d\u0930\u093e\u092a\u094d\u0924}} \u0938\u0947 \u0938\u092e\u094d\u092e\u093e\u0928\u093f\u0924 \u0915\u093f\u092f\u093e \u0917\u092f\u093e | \u0909\u0928\u094d\u0939\u0947\u0902 {{\u0928\u093e\u092e\u093e\u0902\u0915\u093f\u0924 \u0915\u093f\u092f\u093e \u0917\u092f\u093e}} \u0915\u0947 \u0932\u093f\u090f \u0928\u093e\u092e\u093e\u0902\u0915\u093f\u0924 \u092d\u0940 \u0915\u093f\u092f\u093e \u0917\u092f\u093e |", "{{Scientist}} \u0915\u0947 \u092e\u093e\u0924\u093e \u0914\u0930 \u092a\u093f\u0924\u093e {{\u092e\u093e\u0924\u093e}} \u0914\u0930 {{\u092a\u093f\u0924\u093e}} {alivestatus/wgok} \u0914\u0930 \u0909\u0928\u0915\u093e \u0938\u0939\u094b\u0926\u0930 \u092d\u0940 {alivestatus} \u091c\u093f\u0928\u0915\u093e \u0928\u093e\u092e {{\u0938\u0939\u094b\u0926\u0930}} {alivestatus} | \u0909\u0928\u0915\u0947 \u0930\u093f\u0936\u094d\u0924\u0947\u0926\u093e\u0930\u094b\u0902 \u092e\u0947\u0902 {{\u0938\u0902\u092c\u0902\u0927\u0940}} \u092d\u0940 \u0936\u093e\u092e\u093f\u0932 {alivestatus/wgok}|", "\u0909\u0928\u0915\u093e \u0909\u092a\u0928\u093e\u092e {{\u092a\u0930\u093f\u0935\u093e\u0930 \u0915\u093e \u0928\u093e\u092e}} {alivestatus} \u0914\u0930 \u0935\u0939 {{\u092a\u0930\u093f\u0935\u093e\u0930}} \u092a\u0930\u093f\u0935\u093e\u0930 \u0915\u093e \u0939\u093f\u0938\u094d\u0938\u093e \u0939\u094b\u0928\u0947 \u0915\u0947 \u0938\u093e\u0925-\u0938\u093e\u0925 {{\u091c\u093e\u0924\u0940\u092f \u0938\u092e\u0942\u0939}} \u091c\u093e\u0924\u0940\u092f \u0938\u092e\u0942\u0939 \u0915\u0947 {alivestatus/wgok} \u0914\u0930 \u0935\u0939 {{\u0938\u0902\u092c\u0902\u0927\u0928}} \u0938\u0947 \u092d\u0940 {\u091c\u0941\u095c\u0947/\u091c\u0941\u0921\u0940} {alivestatus/wgop} |", "{{Scientist}} {{\u0915\u093e \u092d\u093e\u0917}} \u0915\u093e \u0939\u093f\u0938\u094d\u0938\u093e \u0939\u094b\u0928\u0947 \u0915\u0947 \u0938\u093e\u0925-\u0938\u093e\u0925 {{\u0915\u093e \u0938\u0926\u0938\u094d\u092f}} \u0915\u093e \u0939\u093f\u0938\u094d\u0938\u093e {alivestatus/wgop} \u0914\u0930 \u0907\u0938\u0915\u0947 \u0905\u0932\u093e\u0935\u093e \u0935\u0939 {{\u0930\u093e\u091c\u0928\u0940\u0924\u093f\u0915 \u0926\u0932 \u0915\u0947 \u0938\u0926\u0938\u094d\u092f}} \u0930\u093e\u091c\u0928\u093f\u0924\u093f\u0915 \u0926\u093e\u0932 {\u0915\u0947/\u0915\u0940} \u0938\u0926\u0938\u094d\u092f \u092d\u0940 {alivestatus/wgop} |", "{{Scientist}} {\u0915\u0947/\u0915\u0940} \u091c\u0940\u0935\u0928 \u0938\u093e\u0925\u0940 {{\u091c\u0940\u0935\u0928 \u0938\u093e\u0925\u0940}} {alivestatus/wgok} \u0914\u0930 \u0909\u0928\u0915\u0947 {{\u092c\u091a\u094d\u091a\u094b\u0902 \u0915\u0940 \u0938\u0902\u0916\u094d\u092f\u093e}} {\u092c\u091a\u094d\u091a\u0947/\u092c\u091a\u094d\u091a\u093e} {alivestatus/wgok} \u091c\u093f\u0928\u0915\u093e \u0928\u093e\u092e {{\u0938\u0902\u0924\u093e\u0928}} {alivestatus} |", "{{Scientist}}, \u091c\u093f\u0928\u0915\u0940 \u0928\u093f\u0937\u094d\u0920\u093e {{\u0928\u093f\u0937\u094d\u0920\u093e}} \u0915\u0947 \u0938\u093e\u0925 {alivestatus/wgok}, \u0935\u0939 {{\u0938\u0948\u0928\u094d\u092f \u0936\u093e\u0916\u093e}} \u092e\u0947\u0902 {{\u0938\u0948\u0928\u093f\u0915 \u0938\u094d\u0924\u0930;}} {alivestatus/wgop} \u0914\u0930 \u0909\u0928\u094d\u0939\u094b\u0902\u0928\u0947 {{\u0938\u0902\u0918\u0930\u094d\u0937 \u092e\u0947\u0902 \u092d\u093e\u0917 \u0932\u093f\u092f\u093e}} \u0938\u0902\u0918\u0930\u094d\u0937 \u092e\u0947\u0902 \u092d\u0940 \u092d\u093e\u0917 \u0932\u093f\u092f\u093e {alivestatus} |", "{{Scientist}} \u0915\u0940 \u092e\u0943\u0924\u094d\u092f\u0941 {{\u092e\u0943\u0924\u094d\u092f\u0941 \u0924\u093f\u0925\u093f}} \u0915\u094b {{\u092e\u0943\u0924\u094d\u092f\u0941 \u0915\u093e \u0938\u094d\u0925\u093e\u0928}} \u0926\u0947\u0936 \u092e\u0947\u0902 {{\u092e\u0943\u0924\u094d\u092f\u0941 \u0915\u093e \u0915\u093e\u0930\u0923}} \u0915\u0940 \u0935\u091c\u0939 \u0938\u0947 \u0939\u0941\u0908 \u0914\u0930 \u0909\u0928\u0915\u0947 \u0905\u0902\u0924\u093f\u092e \u0935\u093f\u0936\u094d\u0930\u093e\u092e \u0915\u093e \u0938\u094d\u0925\u093e\u0928 {{\u0905\u0902\u0924\u093f\u092e \u0935\u093f\u0936\u094d\u0930\u093e\u092e \u0915\u093e \u0938\u094d\u0925\u093e\u0928}} \u092e\u0947\u0902 \u0925\u093e |"]
        Properties for Politician:
        selected_keys = [f"{{{{{key}}}}}" for key, value in sorted_dict.items() if value == 0]
selected_keys
selected_keys
['{{संबंधित छवि}}',
 '{{बोले जाने वाली भाषाएँ}}',
 '{{शिक्षित}}',
 '{{विकिमीडिया परियोजना की फोकस सूची पर}}',
 '{{बोली जाने वाली भाषाएं}}',
 '{{भाषाएँ बोली या लिखी या संकेत}}',
 '{{धार्मिक विश्वास या दृष्टिकोण}}',
 '{{कनोनाइजेशन स्थिति}}',
 '{{त्योहार दिवस}}',
 '{{भाषाएँ बोली गईं, लिखी गईं या संकेतित}}',
 '{{कब्र की चित्र}}',
 '{{कब्र स्थान}}',
 '{{संग्रहालय में}}',
 '{{विकिमीडिया प्रोजेक्ट पर फोकस लिस्ट}}',
 '{{बीएचसी यूयूआईडी}}',
 '{{योगदान क्रियेटिव वर्क}}',
 '{{विकिप्रोजेक्ट द्वारा बनाए रखा गया}}',
 '{{बोली गई भाषा}}',
 '{{बंदी स्थल}}',
 '{{कार्य काल (आरम्भ)}}',
 '{{मुंहासे}}',
 '{{यौन अभिमुखता}}',
 '{{विकिमीडिया परियोजना पर फोकस सूची में}}',
 '{{आदरणीय शीर्षक}}',
 '{{मालिक}}',
 '{{सैन्य/पुलिस पद}}',
 '{{इमेज आर्काइव, हर्डर इंस्टीट्यूट}}',
 '{{स्वीडिश ओपन कल्चरल हेरिटेज यूआरआई}}',
 '{{कार्यों में है}}',
 '{{निर्माता के रूप में कॉपीराइट स्थिति}}',
 '{{उत्पत्ति देश}}',
 '{{भाषाएँ बोली लिखी या चिन्हित}}',
 '{{नोबल शीर्षक}}',
 '{{बायोग्राफी एट जर्मन बुंडेस्टैग यूआरएल}}',
 '{{भाषाएँ बोली गई, लिखी गई या संकेतित}}',
 '{{नाम मूलभाषा में}}',
 '{{भाग}}',
 '{{बोले गए, लिखे गए या हस्ताक्षरित भाषाएँ}}',
 '{{मूल्यांकन}}',
 '{{काना में नाम}}',
 '{{विकिमीडिया परियोजना के फोकस सूची पर}}',
 '{{अवरोधन स्थान}}',
 '{{रुचि}}',
 '{{आवासित पद}}',
 '{{अपक्षपात}}',
 '{{अल्पभिन्न}}',
 '{{वर्णन}}',
 '{{संग्रह}}',
 '{{अलग}}',
 '{{डॉक्टरल सलाहक}}',
 '{{भाषाएँ बोली, लिखी या चिह्नित}}',
 '{{लिंग/जेंडर}}',
 '{{रेजेन्सबर्ग वर्गीकरण}}',
 '{{आर्किवएक्स एलओडी}}',
 '{{पितृक}}',
 '{{विकिपीडिया प्रोजेक्ट पर फोकस}}',
 '{{स्मारक छवि}}',
 '{{विकिप्रोजेक्ट द्वारा बनाए रखा}}',
 '{{सिविल रैंक}}',
 '{{हत्यारा}}',
 '{{बीएचसी यूआईडी}}',
 '{{उद्भिद लेखक संक्षिप्ति}}',
 '{{कॉमन्स कैटगरी}}',
 '{{नाम देशज भाषा में}}',
 '{{भाषाएँ बोली जाती है या लिखी जाती है}}',
 '{{शोध निर्देशक}}',
 '{{शोध छात्र}}',
 '{{अलहदा}}',
 '{{रेजेन्सबर्ग श्रेणीकरण}}',
 '{{समाधि छवि}}',
 '{{पत्रिका में}}',
 '{{कॉपीराइट स्थिति}}',
 '{{लेखक संक्षिप्ति (जूलॉजी)}}',
 '{{विकिपीडिया पर ध्यान केंद्रित परियोजना}}',
 '{{भएच यूआईडी}}',
 '{{मूसिक-सैम्लर}}',
 '{{सेज्म-विएल्की}}',
 '{{सिमोगिह}}',
 '{{युग नाम}}',
 '{{पूर्वजों का घर}}',
 '{{प्रसिद्ध कार्य}}',
 '{{मृत्यु की प्रकृति}}',
 '{{शिक्षा संस्थान}}',
 '{{भाषाएँ जिन}}',
 '{{निवास स्थान}}',
 '{{सम्बन्धी}}',
 '{{सृजनात्मक अधिकार की स्थिति}}',
 '{{क्षेत्र}}',
 '{{कॉपीराइट प्रतिनिधि}}',
 '{{सम्बन्धित श्रेणी}}',
 '{{ऑडियो}}',
 '{{म्यूज़िक-सम्लर}}',
 '{{शिक्षक}}',
 '{{बपतिस्मा की तारीख}}',
 '{{विकिमीडिया परियोजनाओं में ध्यान केंद्रित}}',
 '{{सृजनात्मक कॉपीराइट स्थिति}}',
 '{{संबंधित श्रेणी}}',
 '{{विषय की मुख्य टेम्पलेट}}',
 '{{सीओएनओआर}}',
 '{{लास्ट}}',
 '{{बीजीएम}}',
 '{{ई-आर्काइव}}',
 '{{कॉमन्स रचनाकार पृष्ठ}}',
 '{{सैन्य}}',
 '{{प्लाक छवि}}',
 '{{अंतरिक्ष मिशन}}',
 '{{अंतरिक्ष में समय}}',
 '{{विकिमीडिया प्रोजेक्ट की फोकस सूची}}',
 '{{भाषा बोली लिखी या चिन्हित}}',
 '{{शौकिया रेडियो कॉलसाइन}}',
 '{{बोले हुए पाठ ऑडियो}}',
 '{{वीडियो}}',
 '{{पितृक या मातृक}}',
 '{{पैतृक घर}}',
 '{{पूर्वज}}',
 '{{पूर्वज का घर}}',
 '{{उदाहरण है}}',
 '{{बोले, लिखे या हस्ताक्षरित भाषाएँ}}',
 '{{राजनीतिक विचारधारा}}',
 '{{नोस्डेप्यूट.एफआर आइडेंटिफायर}}',
 '{{नोस्डेप्यूट}}',
 '{{धारण की गई स्थिति}}',
 '{{अवधि धारित}}',
 '{{भाषाएँ बोली, लिखी या चिन्हित}}',
 '{{प्रजाति}}',
 '{{बोली गई भाषाएँ}}',
 '{{स्थिति धारण की गई}}',
 '{{लिंग या लिंगिकता}}',
 '{{विवरण यूआरएल}}',
 '{{स्वीडिश संसद व्यक्ति गाइड}}',
 '{{अंतिम विश्राम स्थान}}',
 '{{सैन्य या पुलिस हताहता वर्गीकरण}}',
 '{{भाषाएँ जिन्हें बोली या लिखी गईं}}',
 '{{सम्मानजनक उपसर्ग}}',
 '{{धर्म या विश्वदृष्टिकोण}}',
 '{{इंस्टाग्राम यूजरनेम}}',
 '{{सैन्य मुख्य शाखा}}',
 '{{यू.एस. नेशनल आर्काइव्स आइडेंटिफायर}}',
 '{{भाग लेने वाली}}',
 '{{जन्म स्थल}}',
 '{{नामांकित}}',
 '{{कार्य में}}',
 '{{एनएलसी प्राधिकरण}}',
 '{{प्रोवेनियो यूआईडी}}',
 '{{यौनिक अभिमुखता}}',
 '{{भाग लेने वाले}}',
 '{{यौन अभिविन्यास}}',
 '{{इंस्टाग्राम यूज़रनेम}}',
 '{{जर्मन बुंडेस्टैग के जीवनी यूआरएल}}',
 '{{मैस्टोडॉन पता}}',
 '{{X उपयोक्ता नाम}}',
 '{{बोली गई, लिखी या हस्ताक्षरित भाषाएँ}}',
 '{{फेसबुक उपयोक्ता नाम}}',
 '{{प्रदत्त नाम}}',
 '{{दोषी}}',
 '{{यूएस नेशनल आर्काइव्स आइडेंटिफायर}}',
 '{{भाषाएं बोली या लिखी या संकेत}}',
 '{{कमंस गैलरी}}',
 '{{उदाहरण का}}',
 '{{नाम इन नेटिव लैंग्वेज}}',
 '{{भाषाएँ बोली, लिखी या हस्ताक्षर की}}',
 '{{मूल निवासी भाषा}}',
 '{{पार्लियामेंट}}',
 '{{अभिजात प्रतिष्ठा}}',
 '{{आँखों का रंग}}',
 '{{बालों का रंग}}',
 '{{नागरिक देश}}',
 '{{दूसरा परिवार का नाम स्पेनिश नाम में}}',
 '{{सदस्य है}}',
 '{{जीवनी का बुंडेस्टाग जर्मनी के यूआरएल}}',
 '{{भाषाएँ बोली गई, लिखी या चिन्हित}}',
 '{{आधिकारिक नाम}}',
 '{{फ़ेसबुक यूज़रनेम}}',
 '{{अलग है}}',
 '{{बंदी स्थान}}',
 '{{कुल्तुरस्टिफ्टुंग}}',
 '{{कार्य काल (अंत)}}',
 '{{ब्लैकपास्ट}}',
 '{{अवस्थिति}}',
 '{{भाषाएँ बोली, लिखी या संकेत}}',
 '{{बालक}}',
 '{{बालकों की संख्या}}',
 '{{आरआईए नोवोस्ती संदर्भ}}',
 '{{सैन्य बल}}',
 '{{एक्स username}}',
 '{{फेसबुक username}}',
 '{{इंस्टाग्राम username}}',
 '{{टेलीग्राम username}}',
 '{{देश जहां नागरिकता}}',
 '{{बाल}}',
 '{{उदाहरण का उदाहरण}}',
 '{{सदस्य का सदस्य}}',
 '{{भाषाएं बोली गईं, लिखी गईं या साइन की गईं}}',
 '{{मास्टोडॉन पता}}',
 '{{धार्मिक विश्वास या विश्वदृष्टि}}',
 '{{पार्लेमेंट}}',
 '{{अंतिम संस्कार की तिथि}}',
 '{{अंतिम संस्कार का स्थान}}',
 '{{शिक्षा प्राप्त}}',
 '{{भाषाएँ बोली जाती हैं, लिखी जाती हैं या हस्ताक्षरित}}',
 '{{देश}}',
 '{{भागीदारी}}',
 '{{पीएचडी सलाहक}}',
 '{{दफन स्थान}}',
 '{{भाषाएँ बोली गईं, लिखी गईं या संकेत}}',
 '{{भाषाएँ बोली जाती हैं}}',
 '{{वाचिक नाम}}',
 '{{बालों की शैली/लंबाई}}',
 '{{अंतिम संस्कार की तारीख}}',
 '{{अंतिम संस्कार स्थान}}',
 '{{युद्ध नायक}}',
 '{{पद धारण किया}}',
 '{{पहनावा}}',
 '{{बोली जाने वाली भाषाएँ, लिखित या चिन्हित}}',
 '{{स्पेनिश नाम में दूसरा परिवार का नाम}}',
 '{{मृत्युतिथि}}',
 '{{नाम इन स्वदेशी भाषा}}',
 '{{जेल में डाला गया}}'
        """, # instruction
        "", # input
        "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")
    print("Number of Tokens in Input: " + str(inputs['input_ids'].shape[1]))
    outputs = model.generate(**inputs, max_new_tokens = 4096, use_cache = True)
    print("Number of Tokens in output: " + str(outputs.shape[1]))
    final_output_2 = tokenizer.batch_decode(outputs)
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")

Number of Tokens in Input: 3861
Number of Tokens in output: 4767
Execution time: 83.6291 seconds


In [38]:
final_output_2

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\n        You are a WikiData Expert for Hindi and English. Generate more Template sentences like the ones given in the example below for the Politician occupation.\n        The content between the curly bracket represents the property of wikidata translated into Hindi. Use the Properties for politician to generate similar template sentences.\n        Template Sentences for Scientist:\n        ["{{Scientist}} एक प्रसिद्ध {{व्यवसाय}} {alivestatus/wgop} जिनका जन्म {{जन्म तिथि}} को {{जन्म स्थान}} देश में हुआ {alivestatus} |", "{{नागरिकता}} में पैदा {हुए/हुई} {{Scientist}} {{मातृसंस्था}} {के/की} पूर्व छात्र {alivestatus/wgop} और आगे चलके उन्होंने {{शैक्षिक दर्जा/उपाधि}} की डिग्री भी प्राप्त की |", "{{Scientist}} का कार्यस्थल {{कार्य स्थल}} {alivestatus}, और वह {{नियोक्ता}} में एक {{पद पर आसीन}} क

In [40]:
import re
response = re.search(r"### Response: \n(\[.*?\])", final_output_2[0], re.DOTALL)
if response:
    extracted_response = response.group(1)
    print(extracted_response)

In [41]:
occupation = "politician"
CHROMA_SOURCE = "template_" + occupation
CHROMA_CHUNK_SIZE = 1000
CHROMA_PERSIST_DIRECTORY = "./chroma_db"
CHROMA_COLLECTION_NAME = occupation + "template_files"
EMBEDDINGS_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
start_time = time.time()
import time
start_time = time.time()
target_collection_name = occupation + "_template_files"
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
docs = [Document(page_content=str(extracted_response), metadata={"source": CHROMA_SOURCE})]
text_splitter = CharacterTextSplitter(chunk_size=CHROMA_CHUNK_SIZE, chunk_overlap=0)
docs = text_splitter.split_documents(docs)
print(len(docs))
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
local_chroma = Chroma.from_documents(docs, embeddings, persist_directory=CHROMA_PERSIST_DIRECTORY, collection_name=target_collection_name)
end_time = time.time()

1
